In [1]:
from tkinter import *
import tkinter as tk
from tkinter.ttk import *
from tkinter import messagebox
import tkinter.filedialog as fd #для выбора директории
from PIL import Image, ImageTk #для размещения изображений
import tkcalendar as tkc #для выбора даты
from selenium import webdriver
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
import time
import json
import random
import re
import pandas as pd
import sqlite3
from sqlite3 import Error, IntegrityError
import os 
import datetime
#import rutokenizer as tok
#import rulemma as lem
#import rupostagger as tag
import PyPDF2 as pdf
import pdfminer.high_level
import hashlib

# Тестовые тезаурусы

In [3]:
FZ_223_thesaurus = ['кража', 'терроризм']
FZ_44_thesaurus = ['терроризм', 'кража']

# Запуск приложения

In [6]:
app = tk.Tk()
app.title('ARBITR')
width = 800
height = 650
app.geometry('{}x{}+{}+{}'.format(width, height, (app.winfo_screenwidth() // 2) - (width // 2),\
                                  (app.winfo_screenheight() // 2) - (height // 2)))
app.resizable(width = False, height = False)

#функция открытия окна системного поиска
def system_document_search_window():
    
    def open_document():
        try:
            os.startfile(sys_directory + "/" + str(results_box.get(results_box.curselection())))
        except:
            tk.messagebox.showerror(title = "Ошибка", message = "Документ не был выбран.")
    
    def search_within_directory():
        
        if (len(str(sys_doc_text_entry.get())) == 0) and (len(str(sys_directory_entry.get())) == 0):
                tk.messagebox.showerror(title = "Ошибка", message = "Директория загрузки и ключевое слово или фраза не были введены!\n" + \
                                        "Указанные параметры являются обязательными для запуска приложения.")
        elif (len(str(sys_doc_text_entry.get())) == 0):
            tk.messagebox.showerror(title = "Ошибка", message = "Ключевое слово или фраза не были введены!\n" + \
                                         "Указанный параметр является обязательным для запуска приложения.")
        elif (len(str(sys_directory_entry.get())) == 0):
            tk.messagebox.showerror(title = "Ошибка", message = "Директория загрузки не была введена!\n" + \
                                         "Указанный параметр является обязательным для запуска приложения.")
        else:
            
            try:
                sys_doc_search_button.config(state = 'disabled')

                results_box.delete(0, tk.END)

                def keyword_norm(keyword):

                    t = tok.Tokenizer()
                    t.load()
                    toks = t.tokenize(keyword)

                    tagger = tag.RuPosTagger()
                    tagger.load()
                    tgs = tagger.tag(toks)

                    lemmatizer = lem.Lemmatizer()
                    lemmatizer.load()
                    lemms = lemmatizer.lemmatize(tgs)

                    norm_keywords = []
                    for i in range(len(lemms)):
                        norm_keywords.append(lemms[i][2])

                    return norm_keywords

                def files_to_text(directory):

                    extracted_texts = {}

                    for file in os.listdir(directory):

                        pd_file = open(directory + '/' + file,'rb')

                        text = pdfminer.high_level.extract_text(directory + '/' + file).replace("\n", "").replace("\x0c", "")

                        t = tok.Tokenizer()
                        t.load()
                        tokens = t.tokenize(text)

                        tagger = tag.RuPosTagger()
                        tagger.load()
                        tags = tagger.tag(tokens)

                        lemmatizer = lem.Lemmatizer()
                        lemmatizer.load()
                        lemmas = lemmatizer.lemmatize(tags)

                        parts = []
                        for i in range(len(lemmas)):
                            parts.append(lemmas[i][3])

                        norm_words = []
                        for i in range(len(lemmas)):
                            norm_words.append(lemmas[i][2])

                        extracted_texts[file] = norm_words

                    return extracted_texts

                def calculate_points(extracted_texts, norm_keywords):

                    rate_list = {}

                    for key in extracted_texts:
                        words = extracted_texts[key]
                        positions = []
                        for i in norm_keywords:
                            part = []
                            position = 0
                            for j in words:
                                position += 1
                                if j == i:
                                    part.append(position)
                            positions.append(part)

                        num = 0
                        for i in positions:
                            num += len(i)

                        phrases = 0
                        if len(positions) > 1:
                            for i in positions[0]:
                                tmp = []
                                for j in range(1, len(positions)):
                                    if (i + j) in positions[j]:
                                        tmp.append(True)
                                    else:
                                        tmp.append(False)
                                if False not in tmp:
                                    phrases += 1

                        factor = num + phrases

                        weight = ((num / len(words)) * 100)
                        points = factor * weight

                        if points > 0:
                            rate_list[key] = points

                    return rate_list

                keyword = sys_doc_text_entry.get()
                global sys_directory
                sys_directory = sys_directory_entry.get()
                directory = sys_directory_entry.get()
                norm_keywords = keyword_norm(keyword)
                texts = files_to_text(directory)
                rate_list = calculate_points(texts, norm_keywords)
                if len(rate_list) == 0:
                    result_message.set("Ни одного документа не найдено")
                    sys_doc_search_button.config(state = 'normal')
                    return
                results = sorted(rate_list, key = rate_list.get, reverse = True)
                for result in results:
                    results_box.insert(tk.END, result)
                sys_doc_search_button.config(state = 'normal')
                result_message.set("Документов найдено: " + str(len(results)))
            except:
                tk.messagebox.showerror(title = "Ошибка", message = "Произошла непредвиденная ошибка работы приложения!\n" + \
                                         "Пожалуйста, обратитесь к разработчику.")
    
    #функция, которая позволяет выбрать директорию
    def ask_directory():
        sys_directory_entry.delete(0, tk.END)
        directory = fd.askdirectory()
        sys_directory_entry.insert(0, directory)

    #функция для выполнения очистки формы
    def clear_form():
        sys_doc_text_entry.delete(0, tk.END)
        sys_directory_entry.delete(0, tk.END)
    
    sys_doc_search_win = tk.Toplevel()
    sys_doc_search_win.title("Поиск среди скачанных документов")
    sys_doc_search_win.grab_set()
    width = 650
    height = 550
    sys_doc_search_win.geometry('{}x{}+{}+{}'.format(width, height, (app.winfo_screenwidth() // 2) - (width // 2),\
                                                     (app.winfo_screenheight() // 2) - (height // 2)))
    sys_doc_search_win.resizable(width = False, height = False)
    sys_doc_search_button = tk.Button(sys_doc_search_win, text = "Поиск", width = 12, font = ('bold', 14),\
                                      bg = "#999999", command = search_within_directory)
    sys_doc_search_button.place(x = 390, y = 45, height = 30, anchor = tk.CENTER)
    sys_doc_clear_button = tk.Button(sys_doc_search_win, text = "Очистить", width = 12, font = ('bold', 14),\
                                     bg = "#999999", command = clear_form)
    sys_doc_clear_button. place(x = 570, y = 45, height = 30, anchor = tk.CENTER)
    sys_doc_text_label = tk.Label(sys_doc_search_win, text = 'Ключевое слово или фраза:', font = ('bold', 14))
    sys_doc_text_label.place(x = 10, y = 15, anchor = tk.W)
    sys_doc_text_entry = tk.Entry(sys_doc_search_win, font = ('bold', 12))
    sys_doc_text_entry.place(x = 10, y = 45, width = 300, height = 30, anchor = tk.W)
    sys_directory_label = tk.Label(sys_doc_search_win, text = 'Директория:', font = ('bold', 14))
    sys_directory_label.place(x = 10, y = 75, anchor = tk.W)
    sys_choose_directory_button = tk.Button(sys_doc_search_win, text = "Выбрать", width = 12, font = ('bold', 14),\
                                            bg = "#999999", command = ask_directory)
    sys_choose_directory_button.place(x = 10, y = 105, height = 30, anchor = tk.W)
    sys_directory_entry = tk.Entry(sys_doc_search_win, font = ('bold', 12))
    sys_directory_entry.place(x = 160, y = 105, width = 480, height = 30, anchor = tk.W)
    result_message = tk.StringVar()
    result_info = tk.Label(sys_doc_search_win, textvariable = result_message, font = ('bold', 14))
    result_info.place(x = 10, y = 170, anchor = tk.W)
    open_document_button = tk.Button(sys_doc_search_win, text = "Открыть документ", width = 12, font = ('bold', 14),\
                                     bg = "#999999", command = open_document)
    open_document_button.place(x = 460, y = 170, width = 180, height = 30, anchor = tk.W)
    results_frame = tk.Frame(sys_doc_search_win)
    results_frame.place(x = 10, y = 190, width = 630, height = 350, anchor = tk.NW)
    results_box = tk.Listbox(results_frame)
    results_box.place(x = 0, y = 0, width = 610, height = 350, anchor = tk.NW)
    scroll = tk.Scrollbar(results_frame)
    scroll.config(command = results_box.yview)
    scroll.pack(side = 'right', fill = 'y')
    results_box.config(yscrollcommand = scroll.set, font = ('bold', 14))
    
    
#функция открытия окна администрирования
def administrate_window():
    admin_win = tk.Toplevel()
    admin_win.title("Администрирование")
    admin_win.grab_set()
    width = 700
    height = 700
    admin_win.geometry('{}x{}+{}+{}'.format(width, height, (app.winfo_screenwidth() // 2) - (width // 2),\
                                  (app.winfo_screenheight() // 2) - (height // 2)))
    admin_win.resizable(width = False, height = False)
    
    def create_user():
        
        create_button.config(state = 'disabled')
        
        full_name = FIO_entry.get()
        email = email_entry.get()
        login = username_entry.get()
        raw_password = password_entry.get()
        raw_again_password = again_password_entry.get()
        if ((len(full_name) == 0) or (len(email) == 0) or (len(login) == 0) or (len(raw_password) == 0) or (len(raw_again_password) == 0)):
            tk.messagebox.showerror(title = "Ошибка", message = "Необходимо заполнить все текстовые формы!")
            create_button.config(state = 'normal')
            return
        if choose_role_flag.get() == 0:
            tk.messagebox.showerror(title = "Ошибка", message = "Выберете роль для создаваемого пользователя!")
            create_button.config(state = 'normal')
            return
        passwd = hashlib.md5(password_entry.get().encode('utf8'))
        again_passwd = hashlib.md5(again_password_entry.get().encode('utf8'))
        password = passwd.hexdigest()
        again_password = again_passwd.hexdigest()
        if password != again_password:
            tk.messagebox.showerror(title = "Ошибка", message = "Пароли не совпадают!")
            create_button.config(state = 'normal')
            return
        role_flag = choose_role_flag.get()
        if role_flag == 2:
            creation_ask = tk.messagebox.askyesno(title = "Внимание",\
                                                  message = "Вы пытаетесь создать пользователя с правами администратора.\n"\
                                                  + "Вы хотите продолжить?")
            if creation_ask == False:
                create_button.config(state = 'normal')
                return
        
        if role_flag == 1:
            user_role = 'ПОЛЬЗОВАТЕЛЬ'
        elif role_flag == 2:
            user_role = 'АДМИНИСТРАТОР'
            
        try:
            security_database = 'security_arbitr.db'
            connection = sqlite3.connect(security_database)

            cursor = connection.cursor()
            cursor.execute("""INSERT INTO USER_ROLE_CONN(LOGIN, ROLE_NAME) VALUES (?, ?)""", (login, user_role))
            connection.commit()

            cursor = connection.cursor()
            cursor.execute("""INSERT INTO USERS(FULL_NAME, EMAIL, LOGIN, PASSWORD) VALUES (?, ?, ?, ?)""",\
                           (full_name, email, login, password))
            connection.commit()
            connection.close()
            
            tk.messagebox.showinfo(title = "Внимание", message = "Пользователь успешно создан.\n" +\
                                  "Логин: {}\n".format(login) + "Роль: {}".format(user_role))
            create_button.config(state = 'normal')
            FIO_entry.delete(0, tk.END)
            email_entry.delete(0, tk.END)
            username_entry.delete(0, tk.END)
            password_entry.delete(0, tk.END)
            again_password_entry.delete(0, tk.END)
        except IntegrityError:
            tk.messagebox.showerror(title = "Ошибка", message = "Пользователь с логином {} уже существует.".format(login))
            create_button.config(state = 'normal')
            return
        except:
            tk.messagebox.showerror(title = "Ошибка", message = "Произошла непредвиденная ошибка работы приложения!\n" + \
                                         "Пожалуйста, обратитесь к разработчику.")
            create_button.config(state = 'normal')
            FIO_entry.delete(0, tk.END)
            email_entry.delete(0, tk.END)
            username_entry.delete(0, tk.END)
            password_entry.delete(0, tk.END)
            again_password_entry.delete(0, tk.END)
        
        
    def delete_user():
        
        delete_button.config(state = 'disabled')
        
        login = username_entry.get()
        raw_password = password_entry.get()
        raw_again_password = again_password_entry.get()
        if ((len(login) == 0) or (len(raw_password) == 0) or (len(raw_again_password) == 0)):
            tk.messagebox.showerror(title = "Ошибка", message = "Необходимо заполнить все текстовые формы!")
            delete_button.config(state = 'normal')
            return
        passwd = hashlib.md5(password_entry.get().encode('utf8'))
        again_passwd = hashlib.md5(again_password_entry.get().encode('utf8'))
        password = passwd.hexdigest()
        again_password = again_passwd.hexdigest()
        if password != again_password:
            tk.messagebox.showerror(title = "Ошибка", message = "Пароли не совпадают!\n")
            delete_button.config(state = 'normal')
            return
        try:
            security_database = 'security_arbitr.db'
            connection = sqlite3.connect(security_database)
            cursor = connection.cursor()
            cursor.execute("""SELECT * FROM USERS WHERE LOGIN='{}'""".format(login))
            if len(cursor.fetchall()) == 0:
                tk.messagebox.showerror(title = "Ошибка", message = "Пользователя с логином {} не существует!".format(login))
                delete_button.config(state = 'normal')
                return
            cursor = connection.cursor()
            cursor.execute("""SELECT PASSWORD FROM USERS WHERE LOGIN='{}'""".format(login))
            encrypted_password = cursor.fetchone()[0]
            if encrypted_password != password:
                tk.messagebox.showerror(title = "Ошибка", message = "Введен неверный пароль!")
                delete_button.config(state = 'normal')
                return
            cursor = connection.cursor()
            cursor.execute("""SELECT ROLE_NAME FROM USER_ROLE_CONN WHERE LOGIN='{}'""".format(login))
            deleting_user_role = cursor.fetchone()[0]
            deletion_ask = tk.messagebox.askyesno(title = "Внимание",\
                                                  message = "Вы хотите удалить указанного ниже пользователя?\n" +\
                                  "Логин: {}\n".format(login) + "Роль: {}".format(deleting_user_role))
            if deletion_ask == False:
                delete_button.config(state = 'normal')
                return
            cursor = connection.cursor()
            cursor.execute("""DELETE FROM USER_ROLE_CONN WHERE LOGIN='{}'""".format(login))
            connection.commit()
            cursor = connection.cursor()
            cursor.execute("""DELETE FROM USERS WHERE LOGIN='{}'""".format(login))
            connection.commit()
            connection.close()
            tk.messagebox.showinfo(title = "Внимание", message = "Пользователь {} успешно удален.".format(login))
            delete_button.config(state = 'normal')
            username_entry.delete(0, tk.END)
            password_entry.delete(0, tk.END)
            again_password_entry.delete(0, tk.END)
        except:
            tk.messagebox.showerror(title = "Ошибка", message = "Произошла непредвиденная ошибка работы приложения!\n" + \
                                         "Пожалуйста, обратитесь к разработчику.")
            delete_button.config(state = 'normal')
            username_entry.delete(0, tk.END)
            password_entry.delete(0, tk.END)
            again_password_entry.delete(0, tk.END)
            
    def show_users():
        
        if user_group_flag.get() == 0:
            tk.messagebox.showerror(title = "Ошибка", message = "Выберете категорию пользователей!\n")
        elif user_group_flag.get() == 1:
            security_database = 'security_arbitr.db'
            connection = sqlite3.connect(security_database)
            cursor = connection.cursor()
            cursor.execute("""SELECT LOGIN FROM USER_ROLE_CONN WHERE ROLE_NAME='ПОЛЬЗОВАТЕЛЬ'""")
            result_users = cursor.fetchall()
            connection.close()
            users_box.delete(0, tk.END)
            users_box.insert(tk.END, 'ПОЛЬЗОВАТЕЛИ:')
            users_box.insert(tk.END, '')
            for result_user in result_users:
                users_box.insert(tk.END, result_user)
        elif user_group_flag.get() == 2:
            security_database = 'security_arbitr.db'
            connection = sqlite3.connect(security_database)
            cursor = connection.cursor()
            cursor.execute("""SELECT LOGIN FROM USER_ROLE_CONN WHERE ROLE_NAME='АДМИНИСТРАТОР'""")
            result_users = cursor.fetchall()
            connection.close()
            users_box.delete(0, tk.END)
            users_box.insert(tk.END, 'АДМИНИСТРАТОРЫ:')
            users_box.insert(tk.END, '')
            for result_user in result_users:
                users_box.insert(tk.END, result_user)
            
    
    def select_admin_scenario():
        admin_selector = admin_switcher.get()
        if admin_selector == 1:
            delete_button.config(state = 'normal')
            create_button.config(state = 'disabled')
            FIO_entry.config(state = 'disabled')
            email_entry.config(state = 'disabled')
            choose_user_role.config(state = 'disabled')
            choose_admin_role.config(state = 'disabled')
        elif admin_selector == 0:
            delete_button.config(state = 'disabled')
            create_button.config(state = 'normal')
            FIO_entry.config(state = 'normal')
            email_entry.config(state = 'normal')
            choose_user_role.config(state = 'normal')
            choose_admin_role.config(state = 'normal')
            
    def test_func():
        print(choose_role_flag.get())
        
    def test_func_1():
        if user_group_flag.get() == 0:
            tk.messagebox.showerror(title = "Ошибка", message = "Выберете категорию пользователей!\n")
        else:
            print(user_group_flag.get())
    
    admin_switcher = tk.IntVar()
    admin_switcher.set(0)
    admin_scenario_switcher = tk.Checkbutton(admin_win, text = "Включить режим удаления пользователей",\
                                             variable = admin_switcher, onvalue = 1, offvalue = 0, font = ('bold', 14),\
                                            command = select_admin_scenario)
    admin_scenario_switcher.place(x = 350, y = 20, width = 400, height = 30, anchor = tk.CENTER)
    FIO_label = tk.Label(admin_win, text = 'ФИО пользователя', font = ('bold', 14))
    FIO_label.place(x = 30, y = 70, width = 300, height = 30, anchor = tk.W)
    FIO_entry = tk.Entry(admin_win, font = ('bold', 12))
    FIO_entry.place(x = 30, y = 100, width = 300, height = 30, anchor = tk.W)
    email_label = tk.Label(admin_win, text = 'Адрес электронной почты', font = ('bold', 14))
    email_label.place(x = 30, y = 130, width = 300, height = 30, anchor = tk.W)
    email_entry = tk.Entry(admin_win, font = ('bold', 12))
    email_entry.place(x = 30, y = 160, width = 300, height = 30, anchor = tk.W)
    username_label = tk.Label(admin_win, text = 'Логин', font = ('bold', 14))
    username_label.place(x = 670, y = 70, width = 300, height = 30, anchor = tk.E)
    username_entry = tk.Entry(admin_win, font = ('bold', 12))
    username_entry.place(x = 670, y = 100, width = 300, height = 30, anchor = tk.E)
    password_label = tk.Label(admin_win, text = 'Пароль', font = ('bold', 14))
    password_label.place(x = 670, y = 130, width = 300, height = 30, anchor = tk.E)
    password_entry = tk.Entry(admin_win, font = ('bold', 12), show = '*')
    password_entry.place(x = 670, y = 160, width = 300, height = 30, anchor = tk.E)
    again_password_label = tk.Label(admin_win, text = 'Пароль (повторно)', font = ('bold', 14))
    again_password_label.place(x = 350, y = 190, width = 300, height = 30, anchor = tk.CENTER)
    again_password_entry = tk.Entry(admin_win, font = ('bold', 12), show = '*')
    again_password_entry.place(x = 350, y = 220, width = 300, height = 30, anchor = tk.CENTER)
    choose_role_label = tk.Label(admin_win, text = 'Роль', font = ('bold', 14))
    choose_role_label.place(x = 350, y = 250, width = 300, height = 30, anchor = tk.CENTER)
    global choose_role_flag
    choose_role_flag = tk.IntVar()
    choose_user_role = tk.Radiobutton(admin_win, text = 'Пользователь', value = 1, variable = choose_role_flag, font = ('bold', 14))
    choose_user_role.place(x = 260, y = 280, height = 30, anchor = tk.CENTER)
    choose_admin_role = tk.Radiobutton(admin_win, text = 'Администратор', value = 2, variable = choose_role_flag, font = ('bold', 14))
    choose_admin_role.place(x = 440, y = 280, height = 30, anchor = tk.CENTER)
    create_button = tk.Button(admin_win, text = "Создать", width = 12, font = ('bold', 14), bg = "#999999",\
                              command = create_user)
    create_button.place(x = 200, y = 320, height = 30, anchor = tk.CENTER)
    delete_button = tk.Button(admin_win, text = "Удалить", width = 12, font = ('bold', 14), bg = "#999999",\
                              command = delete_user)
    delete_button.place(x = 500, y = 320, height = 30, anchor = tk.CENTER)
    delete_button.config(state = 'disabled')
    global user_group_flag
    user_group_flag = tk.IntVar()
    users_flag = tk.Radiobutton(admin_win, text = 'Пользователи', value = 1, variable = user_group_flag, font = ('bold', 14))
    users_flag.place(x = 5, y = 380, width = 145, height = 30, anchor = tk.W)
    admins_flag = tk.Radiobutton(admin_win, text = 'Администраторы', value = 2, variable = user_group_flag, font = ('bold', 14))
    admins_flag.place(x = 170, y = 380, width = 170, height = 30, anchor = tk.W)
    show_button = tk.Button(admin_win, text = "Показать", width = 12, font = ('bold', 14), bg = "#999999", command = show_users)
    show_button.place(x = 550, y = 375, height = 30, anchor = tk.W)
    users_frame = tk.Frame(admin_win)
    users_frame.place(x = 10, y = 395, width = 680, height = 295, anchor = tk.NW)
    users_box = tk.Listbox(users_frame)
    users_box.place(x = 0, y = 0, width = 660, height = 300, anchor = tk.NW)
    users_scroll = tk.Scrollbar(users_frame)
    users_scroll.config(command = users_box.yview)
    users_scroll.pack(side = 'right', fill = 'y')
    users_box.config(yscrollcommand = users_scroll.set, font = ('bold', 14))
    
def upload_window():
    
    #здесь будет тело функции, которая реализует поиск и скачивание
    def search_and_upload(doc_text, left_date, right_date, directory, thesaurus = False):

        url = 'http://ras.arbitr.ru/'
        path_to_user_driver = 'C:/Users/user/for_selenium/chromedriver.exe'
        path_to_anticaptcha_plugin = 'C:/Users/user/for_selenium/anticaptcha-plugin_v0.3410.zip'
        anti_captcha_key = 'da54f0f57f07a846c1682eb6e68e04d4'

        date_from = left_date
        date_to = right_date
        doc_text_input = doc_text
        download_directory = directory

        def acp_api_send_request(driver, message_type, data={}):
            message = {
                'receiver': 'antiCaptchaPlugin',
                'type': message_type,
                **data
            }
            return driver.execute_script("""
            return window.postMessage({});
            """.format(json.dumps(message)))

        def init_chrome_options(path_to_anticaptcha_plugin):
            options = webdriver.ChromeOptions()
            options.add_extension(path_to_anticaptcha_plugin)
            options.add_argument('--window-position=5000,5000')
            prefs = {"plugins.always_open_pdf_externally": True, 
                        'download.default_directory': download_directory}
            options.add_experimental_option("prefs",prefs)
            return options

        def get_browser_with_options(path_to_user_driver):
            browser = webdriver.Chrome(path_to_user_driver, options=init_chrome_options(path_to_anticaptcha_plugin) )
            return browser

        def get_initial_page(  url = url, anti_captcha_key = anti_captcha_key  ):
            browser = get_browser_with_options(path_to_user_driver)

            browser.get(url)

            try:
                button_close = browser.find_element_by_class_name('b-promo_notification-popup-close')
                button_close.click()
            except NoSuchElementException:
                pass

            acp_api_send_request(
                browser,
                'setOptions',
                {'options': {'antiCaptchaApiKey': anti_captcha_key}}
            )

            time.sleep(10)

            return browser

        def human_text_enter( element, text_input ):
            time.sleep(random.uniform(2, 5))  
            for letter in text_input: 
                time.sleep(random.uniform(0.05, 0.16)) 
                element.send_keys(letter)

        global initial_browser
        initial_browser = get_initial_page(url = url, anti_captcha_key = anti_captcha_key)

        def get_page_by_input_attributes(browser = initial_browser, doc_text_in = '', date_from_in = '', date_to_in = '' ):

            document_text = browser.find_element_by_xpath('//textarea[@placeholder="текст документа"]')

            human_text_enter(document_text, doc_text_in)

            date_from = browser.find_element_by_xpath('//label[@class="from"]//input[@placeholder="дд.мм.гггг"]')
            date_from.click()
            human_text_enter(date_from, date_from_in)

            date_to = browser.find_element_by_xpath('//label[@class="to"]//input[@placeholder="дд.мм.гггг"]')
            date_to.click()
            human_text_enter(date_to, date_to_in)

            search_button = browser.find_element_by_xpath('//button[@alt="Найти"]')

            WebDriverWait(browser, 120)

            search_button.click()

            #обработка ситуации "Нет результатов"
            time.sleep(random.uniform(8, 10))
            flag = browser.find_elements_by_xpath('//a[@class="b-a-blue js-popupDocumentShow"]')
            if len(flag) == 0:
                browser.close()
                return 0, 0

            else:
                time.sleep(10)

                doc_amount = browser.find_element_by_xpath('//span[@class="b-found"]')
                amount_message = str(doc_amount.text).split()
                for i in amount_message:
                    if i.isdigit():
                        amount = int(i)

                return browser, amount

        global initial_parsing_page

        initial_parsing_page, number = get_page_by_input_attributes(doc_text_in =  doc_text_input, date_from_in = date_from, date_to_in = date_to)

        if initial_parsing_page == 0:
            if thesaurus == False:
                tk.messagebox.showwarning(title = "Внимание", message = "По вашему запросу не найдено ни одного документа.\n" + \
                                              "Измените критерии поиска и запустите приложение снова.")
            return

        if number > 968:
            msg = """Найдено {} документов, удовлетворяющих заданным критериям.\n\n
            За один запуск приложения может быть скачено не более 1000 последних зарегистрированных документов.\n
            Данное ограничение связано со структурой сайта ras.arbitr.ru.\n
            Если Вы хотите скачать остальные документы, измените период поиска.\n\n
            Вы хотите скачать найденные документы?""".format(number)
        else:
            msg = """Документов, удовлетворяющих заданным критериям, найдено: {}.\n
            Вы хотите скачать найденные документы?""".format(number)
        if thesaurus == False:
            ask = tk.messagebox.askyesno(title = 'ARBITR', message = msg)

            if ask == False:
                initial_parsing_page.close()
                return

        def get_row_links_from_1_page(fist_page=initial_parsing_page):
            links = initial_parsing_page.find_elements_by_xpath("//a[@href]")
            all_links = list()
            for link in links:
                all_links.append(link.get_attribute("href"))
            return all_links

        def get_pdf_links(all_links): 
            pdf_links = list()
            for link in all_links:
                if '.pdf' in link:
                    pdf_links.append(link)
                else:
                    continue
            return pdf_links

        def count_found_documents(intial_page = initial_parsing_page):
            total_count = initial_parsing_page.find_elements_by_xpath('//*[@id="contentHeader"]/span[1]')
            split_text = total_count[0].text.split()
            return int(split_text[1])

        def current_doc_info_text(intial_page, n  ):
            pattern_searh = '//*[@id="b-cases"]/li['
            pattern_searh = pattern_searh + str(n) + ']'
            try:
                current_page = initial_parsing_page.find_elements_by_xpath(pattern_searh)
                description = current_page[0].text
            except:
                NoSuchElementException
                pass
            return description

        def get_doc_attributes(description ):
            ordinal_doc_number = int(description[0 : description.find('.') ])
            start_pos_case_number = description.find('\n')
            end_pos_case_number = description.find('\n', start_pos_case_number + 1 )
            case_number = description[start_pos_case_number + 1 : end_pos_case_number ]
            try:
                date_doc = re.findall("\d{2}.\d{2}.\d{2,4}", description)[1]
            except:
                IndexError
                date_doc = re.findall("\d{2}.\d{2}.\d{2,4}", description)[0]

            court_name = description[end_pos_case_number + 1 : description.find(date_doc)]

            start_doc_pos = description.find(date_doc) + 10
            doc_name = description[ start_doc_pos : ]
            indexs = [i for i, symb in enumerate(doc_name) if symb=='\n']
            document_name = ''
            for i in range(len(doc_name)):
                if i not in indexs:
                    document_name +=  doc_name[i] 

            return ordinal_doc_number, case_number, date_doc, court_name, document_name

        def download_description(browser, pdf_links ):

            insert_values = { 'ordinal_number' : [], 
                                'case_number' : [],
                                'date_doc' : [], 
                                'court_name' : [],
                                'document_name' : [],
                                'pdf_link' : [],
                            }

            cnt_documents = count_found_documents()

            if cnt_documents > 40 * 25:
                cnt_documents = 40 * 25 

            for i in range(len(pdf_links)):

                description = current_doc_info_text(browser, i + 1 )

                ordinal_doc_number, case_number, date_doc, court_name, document_name = get_doc_attributes(description = description)

                if pdf_links[i] not in insert_values['pdf_link']:
                    insert_values['ordinal_number'].append(ordinal_doc_number)
                    insert_values['case_number'].append(case_number)
                    insert_values['date_doc'].append(date_doc)
                    insert_values['court_name'].append(court_name)
                    insert_values['document_name'].append(document_name)
                    insert_values['pdf_link'].append(pdf_links[i])
                else:
                    continue
            return insert_values

        def parse_all_documents(initial_parsing_page):
            cnt_documents = count_found_documents(initial_parsing_page)
            flag = True
            i = 1
            x_path = '//*[@id="pages"]/li'
            current_page = initial_parsing_page
            links_need_to_be_downloaded = list()
            list_of_insert_values = list()
            while flag:
                    if i < 10:
                        try :
                            next_page = initial_parsing_page.find_element_by_xpath( x_path + '[' + str(i + 2) + ']/a')
                            raw_links = get_row_links_from_1_page(fist_page=initial_parsing_page)
                            pdf_links = get_pdf_links(raw_links)
                            links_need_to_be_downloaded += pdf_links

                            insert_values = download_description(initial_parsing_page, pdf_links)
                            list_of_insert_values.append(insert_values)

                            next_next_page = next_page.click()
                            i += 1 
                            time.sleep(random.uniform(5, 10))         
                        except : 
                            NoSuchElementException
                            flag = False  


                    elif i % 10 == 0:
                        try:
                            if i == 10: 
                                j = 11
                            else :
                                j = 13 

                            next_page = initial_parsing_page.find_element_by_xpath( x_path + '[' + str(j) + ']/a')

                            raw_links = get_row_links_from_1_page(fist_page=initial_parsing_page)
                            pdf_links = get_pdf_links(raw_links)
                            links_need_to_be_downloaded += pdf_links

                            insert_values = download_description(initial_parsing_page, pdf_links)
                            list_of_insert_values.append(insert_values) 

                            next_next_page = next_page.click()
                            i += 1 
                            time.sleep(random.uniform(5, 10))          
                            next_page = initial_parsing_page.find_element_by_xpath(x_path + '[' + str(j + 1 ) + ']/a')
                            next_page.click()
                            time.sleep(random.uniform(5, 10))  

                        except:
                            NoSuchElementException
                            flag = False
                    else:
                        try:
                            next_page = initial_parsing_page.find_element_by_xpath( x_path + '[' + str(i % 10  + 3)  + ']/a')
                            raw_links = get_row_links_from_1_page(fist_page=initial_parsing_page)
                            pdf_links = get_pdf_links(raw_links)
                            links_need_to_be_downloaded += pdf_links

                            insert_values = download_description(initial_parsing_page, pdf_links)

                            list_of_insert_values.append(insert_values)

                            next_page.click()
                            i += 1 
                            time.sleep(random.uniform(5, 10))        
                        except : 
                            NoSuchElementException
                            flag = False
            raw_links = get_row_links_from_1_page(fist_page=initial_parsing_page)
            pdf_links = get_pdf_links(raw_links)
            links_need_to_be_downloaded += pdf_links       
            insert_values = download_description(initial_parsing_page, pdf_links)
            list_of_insert_values.append(insert_values)
            initial_parsing_page.quit() 

            return links_need_to_be_downloaded, list_of_insert_values

        all_needed_links, list_of_insert_values = parse_all_documents(initial_parsing_page=initial_parsing_page)

        res_insert_values = pd.concat([pd.DataFrame(d) for d in list_of_insert_values]).to_dict('list')

        def del_duplicates(res_insert_values):

            unique = list()
            position = 0

            for i in res_insert_values['ordinal_number']:
                if i not in unique and (len(unique) < (len(set(res_insert_values['ordinal_number'])))):
                    unique.append(i)
                else:
                    res_insert_values['ordinal_number'].pop(position)
                    res_insert_values['case_number'].pop(position)
                    res_insert_values['date_doc'].pop(position)
                    res_insert_values['court_name'].pop(position)
                    res_insert_values['document_name'].pop(position)
                    res_insert_values['pdf_link'].pop(position)
                position += 1   

            df = pd.DataFrame.from_dict(res_insert_values)
            return df

        df = del_duplicates(res_insert_values)

        name_db = 'uploaded_docs.db'

        def sql_connection(db_name = name_db):
            con = sqlite3.connect(name_db)
            return con

        con = sql_connection()

        def create_tables(connection = con):
            cursor_obj = con.cursor()
            cursor_obj.execute(""" CREATE TABLE SEARCH_PARAMS(
                                    ID_DOC INTEGER,  
                                    SEARCH_DIRECTION TEXT,
                                    DATE_FROM TEXT, 
                                    DATE_TO TEXT,
                                    DOWNLOAD_DATE TEXT,
                                    LOCATED_IN_DIRECTORY TEXT ) """)
            con.commit()

            cursor_obj = con.cursor()
            cursor_obj.execute(""" CREATE TABLE DOC_INFO(
                                    ID_DOC INTEGER, 
                                    CASE_NUMBER TEXT, 
                                    DATE_ON_SITE TEXT, -- LAST DATE OF UPDATING DOCUMENT ON SITE 
                                    COURT_NAME TEXT, 
                                    DOC_TITLE TEXT, 
                                    PDF_LINK TEXT) """)
            con.commit()

        try :
            create_tables(connection = con) 
        except:
            pass

        def add_constraints_on_tables(connection  = con):
            cursor_obj = con.cursor()
            cursor_obj.execute(""" ALTER TABLE SEARCH_PARAMS ADD CONSTRAINT ..... """) 

            con.commit()

            cursor_obj.execute(""" ALTER TABLE DOC_INFO ADD CONSTRAINT .... """)

            con.commit()

        def download_docs( browser, df,  connection = con):
            cursor_obj = con.cursor()
            n = len(df['ordinal_number'])
            for i in range(len(df['ordinal_number'])):
                browser.get(df['pdf_link'][i]) 

                cursor_obj.execute("""
                INSERT INTO DOC_INFO(
                            ID_DOC, 
                            CASE_NUMBER, 
                            DATE_ON_SITE, -- LAST DATE OF UPDATING DOCUMENT ON SITE 
                            COURT_NAME, 
                            DOC_TITLE, 
                            PDF_LINK)  
                VALUES (?, ?, ?, ?, ?, ?)""", 
                        ( df['ordinal_number'][i],
                            df['case_number'][i],
                            df['date_doc'][i],
                            df['court_name'][i],
                            df['document_name'][i],
                            df['pdf_link'][i]) )

                time.sleep(1)

                now = datetime.datetime.now()

                cursor_obj.execute("""
                INSERT INTO SEARCH_PARAMS(
                            ID_DOC,
                            SEARCH_DIRECTION, 
                            DATE_FROM, 
                            DATE_TO,  
                            DOWNLOAD_DATE, 
                            LOCATED_IN_DIRECTORY)

                VALUES (?, ?, ?, ?, ?, ?)""", 
                        ( df['ordinal_number'][i],
                            doc_text_input,
                            date_from,
                            date_to,
                            now.strftime("%d-%m-%Y %H:%M"),
                            download_directory) )


                time.sleep(random.uniform(5, 10))

                con.commit()

            con.close()
            browser.quit()

        global browser
        browser = webdriver.Chrome(path_to_user_driver, options = init_chrome_options(path_to_anticaptcha_plugin))
        download_docs(browser = browser, df = df,  connection = con)
        if thesaurus == False:
            tk.messagebox.showinfo(title = "Внимание", message = "Документы были успешно скачаны и помещены в директорию: " + \
                                    directory_entry.get())

    #"функция-обертка" для функции search_and_upload, чтобы обрабатывать исключения
    def function(from_date, to_date, direc, text = None, thesaurus = False, words_and_phrases = None):

        if thesaurus == False:
            if (len(str(doc_text_entry.get())) == 0) and (len(str(directory_entry.get())) == 0):
                tk.messagebox.showerror(title = "Ошибка", message = "Директория загрузки и ключевое слово или фраза не были введены!\n" + \
                                        "Указанные параметры являются обязательными для запуска приложения.")
            elif (len(str(doc_text_entry.get())) == 0):
                 tk.messagebox.showerror(title = "Ошибка", message = "Ключевое слово или фраза не были введены!\n" + \
                                         "Указанный параметр является обязательным для запуска приложения.")
            elif (len(str(directory_entry.get())) == 0):
                 tk.messagebox.showerror(title = "Ошибка", message = "Директория загрузки не была введена!\n" + \
                                         "Указанный параметр является обязательным для запуска приложения.")
            else:
                download_message.set("Пожалуйста, подождите - осуществляется поиск и загрузка документов")
                find_button.config(state = 'disabled')
                try:
                    search_and_upload(text, from_date, to_date, direc, thesaurus)
                    find_button.config(state = 'normal')
                    download_message.set("")
                except:
                    tk.messagebox.showerror(title = "Ошибка", message = "Произошла ошибка работы приложения!\n"\
                                            + "Данная ошибка может быть связана с отсутствием подключения к сети Интернет.\n"\
                                            + "Проверьте подключение к Интернету и запустите приложение снова.\n"\
                                            + "В случае, если данная ошибка повториться, обратитесь к разработчику.")
                    try:
                        initial_browser.close()
                    except:
                        pass
                    try:
                        initial_parsing_page.close()
                    except:
                        pass
                    try:
                        browser.close()
                    except:
                        pass
                    download_message.set("")
                    find_button.config(state = 'normal')
                    find_button.update()
        elif thesaurus == True:
            if (len(str(directory_entry.get())) == 0):
                tk.messagebox.showerror(title = "Ошибка", message = "Директория загрузки не была введена!\n" + \
                                        "Указанный параметр является обязательным для запуска приложения.")
            else:
                msg = "Процесс тематического поиска и скачивания документов может оказаться довольно длительным.\n" +\
                "Вы хотите продолжить?"
                ask = tk.messagebox.askyesno(title = 'ARBITR', message = msg)
                if ask == True:
                    download_message.set("Пожалуйста, подождите - осуществляется поиск и загрузка документов")
                    FZ_223_button.config(state = "disabled")
                    FZ_44_button.config(state = "disabled")
                    try:
                        for combination in words_and_phrases:
                            search_and_upload(combination, from_date, to_date, direc, thesaurus)
                        download_message.set("")
                        tk.messagebox.showinfo(title = "Внимание", message = "Документы были успешно скачаны и помещены в директорию: " + \
                                    directory_entry.get())
                        FZ_223_button.config(state = "normal")
                        FZ_44_button.config(state = "normal")
                    except:
                        tk.messagebox.showerror(title = "Ошибка", message = "Произошла ошибка работы приложения!\n"\
                                                + "Данная ошибка может быть связана с отсутствием подключения к сети Интернет.\n"\
                                                + "Проверьте подключение к Интернету и запустите приложение снова.\n"\
                                                + "В случае, если данная ошибка повториться, обратитесь к разработчику.")
                        try:
                            initial_browser.close()
                        except:
                            pass
                        try:
                            initial_parsing_page.close()
                        except:
                            pass
                        try:
                            browser.close()
                        except:
                            pass
                        download_message.set("")
                        FZ_223_button.config(state = "normal")
                        FZ_44_button.config(state = "normal")
    
    def select_scenario():
        selector = switcher.get()
        if selector == 1:
            doc_text_entry.config(state = 'disabled')
            find_button.config(state = 'disabled')
            FZ_223_button.config(state = 'normal')
            FZ_44_button.config(state = 'normal')
        elif selector == 0:
            doc_text_entry.config(state = 'normal')
            find_button.config(state = 'normal')
            FZ_223_button.config(state = 'disabled')
            FZ_44_button.config(state = 'disabled')

    #функция, которая позволяет выбрать директорию
    def ask_directory():
        directory_entry.delete(0, tk.END)
        directory = fd.askdirectory()
        directory_entry.insert(0, directory)

    #функция для выполнения очистки формы
    def clear_form():
        doc_text_entry.delete(0, tk.END)
        data_from_entry.delete(0, 'end')
        data_to_entry.delete(0, 'end')
        directory_entry.delete(0, tk.END)
    
    upload_win = tk.Toplevel()
    upload_win.title("Скачивание документов")
    upload_win.grab_set()
    width = 720
    height = 490
    upload_win.geometry('{}x{}+{}+{}'.format(width, height, (app.winfo_screenwidth() // 2) - (width // 2),\
                                             (app.winfo_screenheight() // 2) - (height // 2)))
    #переключатель режима работы
    switcher = tk.IntVar()
    switcher.set(0)
    scenario_switcher = tk.Checkbutton(upload_win, text = "Включить режим тематического поиска", variable = switcher,\
                                       onvalue = 1, offvalue = 0, font = ('bold', 14), command = select_scenario)
    scenario_switcher.place(x = 360, y = 40, width = 400, height = 30, anchor = tk.CENTER)

    #кнопка для очистки формы
    clear_button = tk.Button(upload_win, text = "Очистить", width = 12, font = ('bold', 14), command = clear_form, bg = "#999999")
    clear_button.place(x = 360, y = 80, width = 150, height = 30, anchor = tk.CENTER)

    #форма для текста документа
    doc_text_label = tk.Label(upload_win, text = 'Ключевое слово или фраза *', font = ('bold', 14))
    doc_text_label.place(x = 160, y = 120, anchor = tk.CENTER)
    doc_text = tk.StringVar()
    doc_text_entry = tk.Entry(upload_win, font = ('bold', 12), textvariable = doc_text)
    doc_text_entry.place(x = 160, y = 150, width = 300, height = 30, anchor = tk.CENTER)

    #форма для тематического поиска
    search_topic_label = tk.Label(upload_win, text = 'Тема поиска', font = ('bold', 14))
    search_topic_label.place(x = 560, y = 120, anchor = tk.CENTER)
    FZ_223_button = tk.Button(upload_win, text = "223-ФЗ", width = 12, font = ('bold', 14), command = lambda:\
                              function(data_from_entry.get(), data_to_entry.get(),\
                                       directory_entry.get().replace('/', '\\'), text = None,\
                                       thesaurus = True, words_and_phrases = FZ_223_thesaurus), bg = "#999999")
    FZ_223_button.place(x = 480, y = 150, height = 30, anchor = tk.CENTER)
    FZ_223_button.config(state = 'disabled')
    FZ_44_button = tk.Button(upload_win, text = "44-ФЗ", width = 12, font = ('bold', 14), command = lambda:\
                              function(data_from_entry.get(), data_to_entry.get(),\
                                       directory_entry.get().replace('/', '\\'), text = None,\
                                       thesaurus = True, words_and_phrases = FZ_44_thesaurus), bg = "#999999")
    FZ_44_button.place(x = 640, y = 150, height = 30, anchor = tk.CENTER)
    FZ_44_button.config(state = 'disabled')

    #форма для периода поиска
    period_label = tk.Label(upload_win, text = 'Период поиска', font = ('bold', 14))
    period_label.place(x = 360, y = 200, anchor = tk.CENTER)
    #форма для левой границы даты
    data_from_label = tk.Label(upload_win, text = 'с', font = ('bold', 14))
    data_from_label.place(x = 165, y = 230, anchor = tk.CENTER)
    data_from_entry = tkc.DateEntry(upload_win, font = ('bold', 12), width = 12, date_pattern = 'dd.mm.yyyy')
    data_from_entry.delete(0, 'end')
    data_from_entry.place(x = 240, y = 230, height = 30, anchor = tk.CENTER)
    #форма для правой границы даты
    data_to_label = tk.Label(upload_win, text = 'по', font = ('bold', 14))
    data_to_label.place(x = 400, y = 230, anchor = tk.CENTER)
    data_to_entry = tkc.DateEntry(upload_win, font = ('bold', 12), width = 12, date_pattern = 'dd.mm.yyyy')
    data_to_entry.delete(0, 'end')
    data_to_entry.place(x = 480, y = 230, height = 30, anchor = tk.CENTER)

    #форма для директории
    directory_label = tk.Label(upload_win, text = 'Директория загрузки *', font = ('bold', 14))
    directory_label.place(x = 360, y = 280, anchor = tk.CENTER)
    directory_text = tk.StringVar()
    directory_entry = tk.Entry(upload_win, font = ('bold', 12), textvariable = directory_text)
    directory_entry.place(x = 360, y = 310, width = 300, height = 30, anchor = tk.CENTER)
    choose_button = tk.Button(upload_win, text = "Выбрать", width = 12, font = ('bold', 14), command = ask_directory, bg = "#999999")
    choose_button.place(x = 135, y = 310, height = 30, anchor = tk.CENTER)

    #кнопка "Найти"
    find_button = tk.Button(upload_win, text = "Найти и скачать", width = 12, font = ('bold', 14), command = \
                            lambda: function(data_from_entry.get(), data_to_entry.get(),\
                                             directory_entry.get().replace('/', '\\'), doc_text_entry.get(),\
                                             thesaurus = False),bg = "#999999")
    find_button.place(x = 360, y = 390, width = 150, height = 60, anchor = tk.CENTER)

    #текстовое поле, уведомляющее о текущей загрузке
    download_message = tk.StringVar()
    download_info = tk.Label(upload_win, textvariable = download_message, font = ('bold', 14))
    download_info.place(x = 360, y = 540, width = 650, height = 30, anchor = tk.CENTER)
    
#функция для авторизации
def authorize():
    
    if ((len(login_string_entry.get()) == 0) or (len(password_string_entry.get()) == 0)):
        tk.messagebox.showerror(title = "Ошибка", message = "Необходимо ввести логин и пароль!")
        return
    auth_login = login_string_entry.get()
    auth_passwd = hashlib.md5(password_string_entry.get().encode('utf8'))
    auth_password = auth_passwd.hexdigest()
    
    security_database = 'security_arbitr.db'
    connection = sqlite3.connect(security_database)
    cursor = connection.cursor()
    cursor.execute("""SELECT PASSWORD FROM USERS WHERE LOGIN='{}'""".format(auth_login))
    try:
        auth_encrypted_password = cursor.fetchone()[0]
    except TypeError:
        tk.messagebox.showerror(title = "Ошибка", message = "Указан неверный логин или пароль!")
        return
    cursor = connection.cursor()
    cursor.execute("""SELECT ROLE_NAME FROM USER_ROLE_CONN WHERE LOGIN='{}'""".format(auth_login))
    auth_user_role = cursor.fetchone()[0]
    connection.close()
    
    if ((auth_password != auth_encrypted_password) or (len(auth_encrypted_password) == 0)):
        tk.messagebox.showerror(title = "Ошибка", message = "Указан неверный логин или пароль!")
    elif ((auth_password == auth_encrypted_password) and (auth_user_role == 'ПОЛЬЗОВАТЕЛЬ')):
        password_string_entry.delete(0, tk.END)
        password_string_entry.config(state = 'disabled')
        login_string_entry.config(state = 'disabled')
        authorize_button.config(state = 'disabled')
        open_upload_window_button.config(state = 'normal')
        open_sys_doc_search_button.config(state = 'normal')
        auth_info.set('Вы успешно авторизованы')
        role_info.set('Роль: ПОЛЬЗОВАТЕЛЬ')
    elif ((auth_password == auth_encrypted_password) and (auth_user_role == 'АДМИНИСТРАТОР')):
        password_string_entry.delete(0, tk.END)
        password_string_entry.config(state = 'disabled')
        login_string_entry.config(state = 'disabled')
        authorize_button.config(state = 'disabled')
        open_upload_window_button.config(state = 'normal')
        open_sys_doc_search_button.config(state = 'normal')
        open_admin_window_button.config(state = 'normal')
        auth_info.set('Вы успешно авторизованы')
        role_info.set('Роль: АДМИНИСТРАТОР')
        
#название приложения
app_name = tk.Label(app, text = 'ARBITR', font = ('bold', 30), justify = tk.CENTER)
app_name.place(x = 400, y = 30, anchor = tk.CENTER)

#форма для авторизации
login_label = tk.Label(app, text = 'Логин', font = ('bold', 14))
login_label.place(x = 400, y = 90, width = 260, height = 30, anchor = tk.CENTER)
login_string_entry = tk.Entry(app, font = ('bold', 14))
login_string_entry.place(x = 400, y = 120, width = 300, height = 30, anchor = tk.CENTER)
password_label = tk.Label(app, text = 'Пароль', font = ('bold', 14))
password_label.place(x = 400, y = 150, width = 260, height = 30, anchor = tk.CENTER)
password_string_entry = tk.Entry(app, font = ('bold', 14), show = '*')
password_string_entry.place(x = 400, y = 180, width = 300, height = 30, anchor = tk.CENTER)
authorize_button = tk.Button(app, text = "Войти", width = 12, font = ('bold', 14), command = authorize, bg = "#999999")
authorize_button.place(x = 400, y = 230, width = 150, height = 30, anchor = tk.CENTER)
auth_info = StringVar()
authorize_info = tk.Label(app, textvariable = auth_info, font = ('bold', 14))
authorize_info.place(x = 400, y = 260, height = 30, anchor = tk.CENTER)
role_info = StringVar()
user_role_info = tk.Label(app, textvariable = role_info, font = ('bold', 14))
user_role_info.place(x = 400, y = 290, height = 30, anchor = tk.CENTER)

open_upload_window_button = tk.Button(app, text = "Скачивание документов", width = 12, font = ('bold', 14),\
                                      command = upload_window, bg = "#999999")
open_upload_window_button.place(x = 400, y = 380, width = 350, height = 40, anchor = tk.CENTER)
#open_upload_window_button.config(state = "disabled")
open_sys_doc_search_button = tk.Button(app, text = "Поиск среди скачанных документов", width = 12, font = ('bold', 14),\
                                       command = system_document_search_window, bg = "#999999")
open_sys_doc_search_button.place(x = 400, y = 460, width = 350, height = 40, anchor = tk.CENTER)
#open_sys_doc_search_button.config(state = "disabled")
open_admin_window_button = tk.Button(app, text = "Администрирование", width = 12, font = ('bold', 14),\
                                     command = administrate_window, bg = "#999999")
open_admin_window_button.place(x = 400, y = 580, width = 350, height = 40, anchor = tk.CENTER)
#open_admin_window_button.config(state = "disabled")

app.mainloop()